In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import math

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

dtypes = {
    'season': np.uint8,
    'holiday': np.uint8,
    'workingday': np.uint8,
    'temp': np.float16,
    'atemp': np.float16,
    'humidity': np.float16,
    'windspeed': np.float16,
    'casual': np.uint16,
    'registered': np.uint16,
    'count': np.uint16
}

df = pd.read_csv('bike_train.csv', dtype=dtypes, parse_dates=['datetime'])
df = df.sample(len(df))
df.head(20)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
8938,2012-08-14 19:00:00,3,0,1,1,31.156250,36.375000,66.0,13.000000,94,533,627
4763,2011-11-11 13:00:00,4,1,0,1,15.578125,19.687500,37.0,37.000000,57,179,236
2520,2011-06-12 22:00:00,2,0,0,1,26.234375,28.796875,83.0,9.000000,31,97,128
5471,2012-01-03 02:00:00,1,0,1,1,7.378906,8.335938,51.0,17.000000,0,3,3
10175,2012-11-09 09:00:00,4,0,1,1,13.937500,15.148438,53.0,28.000000,23,304,327
10253,2012-11-12 15:00:00,4,1,0,1,22.953125,26.515625,64.0,17.000000,102,280,382
2169,2011-05-17 07:00:00,2,0,1,2,21.312500,25.000000,94.0,17.000000,13,138,151
3537,2011-08-17 07:00:00,3,0,1,1,27.062500,31.062500,65.0,6.003906,12,296,308
268,2011-01-12 15:00:00,1,0,1,1,8.203125,8.335938,47.0,28.000000,1,48,49
6332,2012-03-01 02:00:00,1,0,1,2,18.859375,22.718750,94.0,19.000000,0,6,6


In [2]:
X = df.drop(['casual', 'registered', 'count'], axis=1)
y = df[['casual', 'registered']].values

In [3]:
from sklearn.preprocessing import StandardScaler, maxabs_scale, minmax_scale

def preprocess(X):
    # column 추가
    X['year'] = X['datetime'].map(lambda x: np.uint8(x.year))
    X['hour'] = X['datetime'].map(lambda x: np.uint8(x.hour))
    X['hour3'] = X['datetime'].map(lambda x: np.uint8(x.hour / 3))
    X['hour6'] = X['datetime'].map(lambda x: np.uint8(x.hour / 6))
    X['month'] = X['datetime'].map(lambda x: np.uint8(x.month))
    X['month3'] = X['datetime'].map(lambda x: np.uint8(x.month / 3))
    X['weekday'] = X['datetime'].map(lambda x: np.uint8(x.weekday()))
    X['weekend'] = X['weekday'].map(lambda x: 1 if x in [5, 6] else 0)
    
    # one_hot 인코딩
    X = pd.get_dummies(X, columns=['year', 'season', 'hour', 'hour3', 'hour6', 'weather', 'weekday', 'month', 'month3'])
    
    # 필요없는 column 제거
    del X['datetime']
    
    # scaling
    X['temp'] = minmax_scale(X['temp'])
    X['atemp'] = minmax_scale(X['atemp'])
    X['humidity'] = minmax_scale(X['humidity'])
    X['windspeed'] = minmax_scale(X['windspeed'])
    
    return X


processedX = preprocess(X)
processedX.head(10)

,holiday,workingday,temp,atemp,humidity,windspeed,weekend,year_219,year_220,season_1,...,month_8,month_9,month_10,month_11,month_12,month3_0,month3_1,month3_2,month3_3,month3_4
8938,0,1,0.754883,0.796387,0.660156,0.228149,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4763,1,0,0.367188,0.423340,0.370117,0.649414,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
2520,0,0,0.632324,0.626953,0.830078,0.157959,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
5471,0,1,0.163208,0.169556,0.510254,0.298340,0,0,1,1,...,0,0,0,0,0,1,0,0,0,0
10175,0,1,0.326416,0.321777,0.530273,0.491211,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
10253,1,0,0.550781,0.576172,0.640137,0.298340,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
2169,0,1,0.509766,0.541992,0.940430,0.298340,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3537,0,1,0.652832,0.677734,0.649902,0.105347,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
268,0,1,0.183716,0.169556,0.470215,0.491211,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
6332,0,1,0.448730,0.491211,0.940430,0.333496,0,0,1,1,...,0,0,0,0,0,0,1,0,0,0


In [12]:
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Dense, Dropout
from keras.layers.noise import GaussianDropout, AlphaDropout
from keras.optimizers import Adam

model = Sequential()
model.add(Dense(256, input_shape=(77,), activation='relu', bias_initializer='lecun_normal'))
model.add(GaussianDropout(0.5))
model.add(Dense(256, activation='relu', bias_initializer='lecun_normal'))
model.add(GaussianDropout(0.5))
model.add(Dense(256, activation='relu', bias_initializer='lecun_normal'))
model.add(GaussianDropout(0.5))
model.add(Dense(256, activation='relu', bias_initializer='lecun_normal'))
model.add(GaussianDropout(0.5))
model.add(Dense(256, activation='relu', bias_initializer='lecun_normal'))
model.add(GaussianDropout(0.5))
model.add(Dense(2))
model.compile(optimizer=Adam(0.0001), loss='mean_squared_logarithmic_error')

In [14]:
model.fit(processedX.values, y, validation_split=0.1, epochs=100)

Train on 9797 samples, validate on 1089 samples
Epoch 1/100
9797/9797 [==============================] - 2s - loss: 4.6369 - val_loss: 1.9800
Epoch 2/100
9797/9797 [==============================] - 1s - loss: 1.8182 - val_loss: 1.2509
Epoch 3/100
9797/9797 [==============================] - 1s - loss: 1.0429 - val_loss: 0.6632
Epoch 4/100
9797/9797 [==============================] - 1s - loss: 0.7369 - val_loss: 0.6072
Epoch 5/100
9797/9797 [==============================] - 1s - loss: 0.6564 - val_loss: 0.5611
Epoch 6/100
9797/9797 [==============================] - 1s - loss: 0.6180 - val_loss: 0.5212
Epoch 7/100
9797/9797 [==============================] - 1s - loss: 0.5743 - val_loss: 0.4929
Epoch 8/100
9797/9797 [==============================] - 1s - loss: 0.5407 - val_loss: 0.4736
Epoch 9/100
9797/9797 [==============================] - 1s - loss: 0.5152 - val_loss: 0.4339
Epoch 10/100
9797/9797 [==============================] - 1s - loss: 0.4858 - val_loss: 0.4168
Epoch 11/10

9797/9797 [==============================] - 1s - loss: 0.2067 - val_loss: 0.1748
Epoch 87/100
9797/9797 [==============================] - 1s - loss: 0.2079 - val_loss: 0.1766
Epoch 88/100
9797/9797 [==============================] - 1s - loss: 0.2015 - val_loss: 0.1778
Epoch 89/100
9797/9797 [==============================] - 1s - loss: 0.2072 - val_loss: 0.1743
Epoch 90/100
9797/9797 [==============================] - 1s - loss: 0.2060 - val_loss: 0.1840
Epoch 91/100
9797/9797 [==============================] - 1s - loss: 0.2036 - val_loss: 0.1759
Epoch 92/100
9797/9797 [==============================] - 1s - loss: 0.2047 - val_loss: 0.1806
Epoch 93/100
9797/9797 [==============================] - 1s - loss: 0.2041 - val_loss: 0.1800
Epoch 94/100
9797/9797 [==============================] - 1s - loss: 0.2020 - val_loss: 0.1808
Epoch 95/100
9797/9797 [==============================] - 1s - loss: 0.2003 - val_loss: 0.1849
Epoch 96/100
9797/9797 [==============================] - 1s - 

In [15]:
df_test = pd.read_csv('bike_test.csv', dtype=dtypes, parse_dates=['datetime'])
X_test = preprocess(df_test)

In [16]:
df_test['count'] = np.sum(model.predict(X_test.values), axis=1)

In [17]:
df_test[['datetime', 'count']].to_csv('output.csv', index=False)